In [4]:
import pandas as pd 
import numpy as np
import csv
from interpret.glassbox import ExplainableBoostingClassifier
from interpret import show
from sklearn.metrics import roc_auc_score

#### Load Data -- Continuous

In [7]:
### load data
data = pd.read_csv("~/Documents/Duke/Cynthia Research/KY-analysis-mytrials/KY Recidivism/KY data/kentucky_data.csv")
data = data.drop(['person_id', 'screening_date', 'race', 'fta_risk_score_raw','nca_risk_score_raw',
                  'pvf_risk_score_raw', 'fta_calc', 'nca_calc', 'pvf_calc'], axis=1)
data = data[data['p_charges'] <= 20]
X = data.loc[:,:'current_pending_charge'].values
Y = data['recid_two_year'].values
cols = data.columns[:-14]

In [10]:
ebm = ExplainableBoostingClassifier(feature_names=cols, 
                                    n_estimators=20, 
                                    max_tree_splits=3, 
                                    learning_rate=0.1, 
                                    holdout_split=0.15, 
                                    random_state=816).fit(X, Y)

In [11]:
Y_pred = ebm.predict_proba(X)[:,1]
roc_auc_score(Y, Y_pred)

0.7294176110483268

In [13]:
ebm_global = ebm.explain_global()
show(ebm_global)

In [46]:
#log model results to the model performance folder, as per standards
path = "C:\\Users\\binha\\Documents\\Duke\\Cynthia Research\\KY-analysis-mytrials\\KY Recidivism\\KY Results\\Models\\Six Month\\"

train_auc_mean, train_auc_std = np.mean(train_auc), np.std(train_auc)
test_auc_mean, test_auc_std = np.mean(test_auc), np.std(test_auc)
                   
results = [["Model", "train_auc_mean", "train_auc_std", "validation_auc_mean", "validation_auc_std", "holdout_test_auc", "holdout_test_acc"],
    ["Logistic Regression", train_auc_mean, train_auc_std, test_auc_mean, test_auc_std, heldout_test_auc, heldout_test_acc ]]

with open(path + 'GAM.csv', 'w') as writeFile:
    writer = csv.writer(writeFile)
    writer.writerows(results)